<a href="https://colab.research.google.com/github/theAkhileshRai/Projects/blob/master/XGB_Multi_and_Binary_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
df1 = pd.read_csv('/content/Combined_df.csv',index_col=False)

In [114]:
df1.head()

,Unnamed: 0,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,y_true,leak_level,station
0,0,05/04/19 2:30,1,0.666534,2.080444,3.220836,10.105964,10.063987,0.666534,2.053527,3.307406,10.142075,10.063987,1.0,2.0,Cross Barr
1,1,05/04/19 3:30,1,0.716066,2.153494,3.135203,9.998847,10.075669,0.716066,2.120434,3.243347,10.053597,10.075669,1.0,2.0,Cross Barr
2,2,05/04/19 4:30,1,0.687551,2.142630,3.066270,9.954641,10.081478,0.687551,2.105328,3.181119,10.001306,10.081478,1.0,2.0,Cross Barr
3,3,05/04/19 5:30,1,0.686916,2.140013,3.071850,9.956670,10.054274,0.686916,2.101800,3.191233,10.008420,10.054274,1.0,2.0,Cross Barr
4,4,05/04/19 6:30,1,0.698125,2.156244,3.063865,9.961413,10.091478,0.698125,2.117328,3.183534,10.009700,10.091478,1.0,2.0,Cross Barr


In [115]:
df1.columns

Index(['Unnamed: 0', 'time', 'cyl', 's_temp_crank', 'd_temp_crank',
       's_pressure_crank', 'd_pressure_crank', 'rpm_crank', 's_temp_mean',
       'd_temp_mean', 's_pressure_mean', 'd_pressure_mean', 'rpm_mean',
       'y_true', 'leak_level', 'station'],
      dtype='object')

In [0]:
df1 = df1.drop(columns='Unnamed: 0')

In [117]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,y_true,leak_level,station
0,05/04/19 2:30,1,0.666534,2.080444,3.220836,10.105964,10.063987,0.666534,2.053527,3.307406,10.142075,10.063987,1.0,2.0,Cross Barr
1,05/04/19 3:30,1,0.716066,2.153494,3.135203,9.998847,10.075669,0.716066,2.120434,3.243347,10.053597,10.075669,1.0,2.0,Cross Barr
2,05/04/19 4:30,1,0.687551,2.142630,3.066270,9.954641,10.081478,0.687551,2.105328,3.181119,10.001306,10.081478,1.0,2.0,Cross Barr
3,05/04/19 5:30,1,0.686916,2.140013,3.071850,9.956670,10.054274,0.686916,2.101800,3.191233,10.008420,10.054274,1.0,2.0,Cross Barr
4,05/04/19 6:30,1,0.698125,2.156244,3.063865,9.961413,10.091478,0.698125,2.117328,3.183534,10.009700,10.091478,1.0,2.0,Cross Barr


In [118]:
df1.shape

(165861, 15)

In [0]:
X = df1.iloc[:,1:12]

In [120]:
X

,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean
0,1,0.666534,2.080444,3.220836,10.105964,10.063987,0.666534,2.053527,3.307406,10.142075,10.063987
1,1,0.716066,2.153494,3.135203,9.998847,10.075669,0.716066,2.120434,3.243347,10.053597,10.075669
2,1,0.687551,2.142630,3.066270,9.954641,10.081478,0.687551,2.105328,3.181119,10.001306,10.081478
3,1,0.686916,2.140013,3.071850,9.956670,10.054274,0.686916,2.101800,3.191233,10.008420,10.054274
4,1,0.698125,2.156244,3.063865,9.961413,10.091478,0.698125,2.117328,3.183534,10.009700,10.091478
...,...,...,...,...,...,...,...,...,...,...,...
165856,4,0.728417,2.599025,0.331039,2.487713,12.358284,0.728417,2.854156,0.184073,2.375134,12.358284
165857,4,0.686059,2.556454,0.326915,2.484471,12.359520,0.686059,2.810078,0.179784,2.370367,12.359520
165858,4,0.663322,2.530006,0.328148,2.487526,12.352143,0.663322,2.785113,0.180072,2.373662,12.352143
165859,4,0.645405,2.513933,0.325451,2.486730,12.353393,0.645405,2.767917,0.178354,2.373709,12.353393


In [0]:
y_true = df1.iloc[:,13]

In [0]:
y_true=y_true.fillna(0)

In [163]:
y_true.unique()

array([2., 0., 3., 1., 4.])

In [0]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size)

In [165]:
print(X_train.shape,y_train.shape)

(111126, 11) (111126,)


In [166]:
X_test.shape

(54735, 11)

In [167]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [168]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = model.predict(X_test)

In [0]:
predictions = [round(value) for value in y_pred]

In [171]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.27%


In [0]:
from sklearn.metrics import multilabel_confusion_matrix

In [173]:
multilabel_confusion_matrix(y_test,predictions,labels=[0,1,2,3,4])

array([[[ 3590,  5854],
        [  414, 44877]],

       [[53017,    64],
        [ 1309,   345]],

       [[50680,   122],
        [ 2347,  1586]],

       [[53405,    31],
        [  729,   570]],

       [[51825,   352],
        [ 1624,   934]]])

In [0]:
import pickle
filename = '/content/finalized_model.pkl'


In [0]:
pickle.dump(model, open(filename, 'wb'))

In [136]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8857769251849822


In [177]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,target_names = ['0','1','2','3','4']))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93     45291
           1       0.84      0.21      0.33      1654
           2       0.93      0.40      0.56      3933
           3       0.95      0.44      0.60      1299
           4       0.73      0.37      0.49      2558

    accuracy                           0.88     54735
   macro avg       0.87      0.48      0.58     54735
weighted avg       0.88      0.88      0.86     54735



# Binary Classifier

In [0]:
binary_true = df1.iloc[:,12]

In [0]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, binary_true, test_size=test_size)

In [148]:
binary_true.unique()

array([1., 0.])

In [149]:
model = XGBClassifier(learning_rate=0.05,max_delta_step=1,n_estimators=200)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=1, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = model.predict(X_test)

In [0]:
binary_predictions = [round(value) for value in y_pred]

In [152]:
accuracy = accuracy_score(y_test, binary_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.73%


In [153]:
multilabel_confusion_matrix(y_test,binary_predictions,labels=[0,1])

array([[[ 2725,  6897],
        [  367, 44746]],

       [[44746,   367],
        [ 6897,  2725]]])

In [0]:
import pickle
filename = '/content/binary_finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [155]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8672878414177401


In [158]:
from sklearn.metrics import classification_report
print(classification_report(y_test, binary_predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.99      0.92     45113
         1.0       0.88      0.28      0.43      9622

    accuracy                           0.87     54735
   macro avg       0.87      0.64      0.68     54735
weighted avg       0.87      0.87      0.84     54735

